In [1]:
import os
import os.path as osp
import numpy as np
import pandas as pd
import argparse

from typing import Tuple, List, Dict, Any

from ukge.datasets import KGTripleDataset
from ukge.losses import compute_det_transe_distmult_loss, compute_det_complex_loss, compute_det_rotate_loss
from ukge.metrics import Evaluator


from typing import Tuple, List, Dict, Any
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Embedding

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

model = 'distmult'
dataset = 'cn15k'
confidence_score_function = 'logi'
hidden_dim = 128
num_neg_per_positive = 10
batch_size = 1024
lr = 0.01
weight_decay = 0.005

model_checkpoint = '/home/mou/Projects/UKGE-FL/results/unc_cn15k_distmult_confi_logi/lr_0.01_wd_0.0_dim_128/best_model_ndcg_lin_topk.pth'

In [ ]:
class KGEModel(torch.nn.Module):
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        hidden_channels: int,
        sparse: bool = False
    ):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_relations = num_relations
        self.hidden_channels = hidden_channels
        self.node_emb = Embedding(num_nodes, hidden_channels, sparse=sparse)
        self.rel_emb = Embedding(num_relations, hidden_channels, sparse=sparse)
        self.fc = torch.nn.Sequential(
            # torch.nn.Linear(1, 1, bias=True),
            # torch.nn.Linear(1, 32, bias=True),
            # torch.nn.ReLU(inplace=True),
            # torch.nn.Linear(32, 32, bias=False),
            # torch.nn.ReLU(inplace=True),
            # torch.nn.Linear(32, 1, bias=False),
            torch.nn.Sigmoid()
        )

    def reset_parameters(self):
        self.node_emb.reset_parameters()
        self.rel_emb.reset_parameters()

    def forward(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:
        raise NotImplementedError
    
    def get_embeddings(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ):
        raise NotImplementedError
    
    def get_plausibility_score(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:
        raise NotImplementedError

    def get_confidence_score(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:
        raise NotImplementedError
    
class DistMult(KGEModel):
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        hidden_channels: int,
        sparse: bool = False,
        confidence_score_function: str = "logi",
    ):
        super().__init__(num_nodes, num_relations, hidden_channels, sparse)
        self.confidence_score_function = confidence_score_function
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.node_emb.weight)
        torch.nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:
        head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)
        tail = self.node_emb(tail_index)
        return (head * rel * tail).sum(dim=-1)

    def get_embeddings(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tuple[Tensor, Tensor, Tensor]:
        head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)
        tail = self.node_emb(tail_index)
        return head, rel, tail
    
    def get_confidence_score(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor
    ) -> Tensor:
        head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)
        tail = self.node_emb(tail_index)
        plausibility_score = (head * rel * tail).sum(dim=-1)
        if self.confidence_score_function == "logi":
            return torch.sigmoid(self.fc(plausibility_score.view(-1, 1)).view(-1))
        elif self.confidence_score_function == "rect":
            return torch.clamp(self.fc(plausibility_score.view(-1, 1)).view(-1), min=0, max=1)

    def get_plausibility_score(
        self,
        head_index: Tensor,
        rel_type: Tensor,
        tail_index: Tensor,
    ) -> Tensor:
        return self.forward(head_index, rel_type, tail_index)

In [2]:
train_dataset = KGTripleDataset(dataset=dataset, split='train', num_neg_per_positive=num_neg_per_positive, deterministic=False)
val_dataset = KGTripleDataset(dataset=dataset, split='val')
test_dataset = KGTripleDataset(dataset=dataset, split='test')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DistMult(num_nodes=train_dataset.num_cons(), num_relations=train_dataset.num_rels(), hidden_channels=hidden_dim, confidence_score_function=confidence_score_function)

model.load_state_dict(torch.load(model_checkpoint)['state_dict'])
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=weight_decay)


test_evaluator = Evaluator(test_dataloader, model, batch_size=batch_size, device=device)


In [3]:
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = next(iter(train_dataloader))
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = pos_hrt.long(), pos_target.float(), neg_hn_rt.long(), neg_hr_tn.long()
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = pos_hrt.to(device), pos_target.to(device), neg_hn_rt.to(device), neg_hr_tn.to(device) 
neg_hn_rt = neg_hn_rt.view(-1, neg_hn_rt.size(-1))
neg_hr_tn = neg_hr_tn.view(-1, neg_hr_tn.size(-1))

optimizer.zero_grad()
pred_pos_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
pred_neg_hn_score = model.get_confidence_score(neg_hn_rt[:, 0], neg_hn_rt[:, 1], neg_hn_rt[:, 2])
pred_neg_tn_score = model.get_confidence_score(neg_hr_tn[:, 0], neg_hr_tn[:, 1], neg_hr_tn[:, 2])
neg_target = torch.zeros_like(pred_neg_hn_score)

In [4]:
print('pos_target:', pos_target)
print('pred_pos_score:', pred_pos_score)
print('pred_neg_hn_score:', pred_neg_hn_score)
print('pred_neg_tn_score:', pred_neg_tn_score)

pos_target: tensor([0.8927, 0.7093, 0.7093,  ..., 0.4727, 0.7093, 0.7093], device='cuda:0')
pred_pos_score: tensor([0.5045, 0.5045, 0.5045,  ..., 0.5045, 0.5045, 0.5045], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
pred_neg_hn_score: tensor([0.5045, 0.5045, 0.5045,  ..., 0.5045, 0.5045, 0.5045], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
pred_neg_tn_score: tensor([0.5045, 0.5045, 0.5045,  ..., 0.5045, 0.5045, 0.5045], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [5]:
pos_loss = criterion(pred_pos_score, pos_target)
neg_loss = (criterion(pred_neg_hn_score, neg_target) + criterion(pred_neg_tn_score, neg_target)) / 2
loss = pos_loss + neg_loss
loss.backward()
optimizer.step()

In [6]:

pred_pos_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
pred_neg_hn_score = model.get_confidence_score(neg_hn_rt[:, 0], neg_hn_rt[:, 1], neg_hn_rt[:, 2])
pred_neg_tn_score = model.get_confidence_score(neg_hr_tn[:, 0], neg_hr_tn[:, 1], neg_hr_tn[:, 2])

In [7]:
print('pos_target:', pos_target)
print('pred_pos_score:', pred_pos_score)

pos_target: tensor([0.8927, 0.7093, 0.7093,  ..., 0.4727, 0.7093, 0.7093], device='cuda:0')
pred_pos_score: tensor([0.5029, 0.5029, 0.5029,  ..., 0.5029, 0.5029, 0.5029], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [8]:
model_state_dict = model.state_dict()
for key in model_state_dict.keys():
    print(key)
    print(model_state_dict[key])

node_emb.weight
tensor([[ 1.9253e-08, -2.6548e-08, -1.0348e-08,  ..., -3.0833e-08,
          2.8742e-08,  1.7306e-08],
        [ 2.4741e-08,  1.2975e-08, -4.0982e-08,  ..., -2.5963e-08,
         -4.7117e-08, -1.5245e-09],
        [-2.4240e-08, -1.7536e-09, -6.4314e-10,  ..., -1.4605e-09,
          3.6167e-08,  4.3762e-08],
        ...,
        [-4.1555e-08, -4.0360e-08,  4.8920e-08,  ...,  3.4273e-08,
         -2.5535e-09,  1.9661e-08],
        [ 4.3286e-08, -1.6908e-08, -3.3596e-08,  ..., -7.0039e-08,
         -3.4603e-10,  3.0957e-08],
        [-1.8841e-08, -1.2777e-08,  1.5567e-08,  ...,  3.6009e-08,
          3.6712e-08,  2.8677e-09]], device='cuda:0')
rel_emb.weight
tensor([[-2.6009e-08, -6.7773e-07, -4.5543e-07,  ...,  2.1239e-07,
          3.9273e-07,  6.0972e-07],
        [ 4.8184e-08, -4.4124e-07,  4.1636e-07,  ..., -4.3142e-07,
         -1.2952e-07, -1.9767e-09],
        [-2.8186e-07, -2.3445e-08,  4.4095e-07,  ..., -6.4428e-07,
          3.4258e-09, -3.3607e-07],
        ...

In [9]:
model.eval()
test_evaluator.update_hr_scores_map()
test_mean_ndcg = test_evaluator.get_mean_ndcg()
test_mse = test_evaluator.get_mse()
test_mae = test_evaluator.get_mae()

Updating hr_scores_map...


100%|██████████| 8063/8063 [02:08<00:00, 62.70it/s]


In [10]:
print(test_mse)
print(test_mae)
print(test_mean_ndcg[0])
print(test_mean_ndcg[1])

0.1155225036682467
0.3899015522221222
0.1352762755276738
0.13376590751617082


In [11]:
hrtw_map = test_evaluator.hrtw_map
hr_scores_map = test_evaluator.hr_scores_map

In [12]:
hrt = [[h, r, t, hrtw_map[h][r][t], hr_scores_map[h][r][t]] for h in hrtw_map.keys() for r in hrtw_map[h].keys() for t in hrtw_map[h][r].keys()]
hrt_df = pd.DataFrame(hrt, columns=['head', 'tail', 'relation', 'score', 'prediction'])

In [13]:
hrt_df.to_csv('hrt_df.csv', index=False)

In [14]:
pos_hrt, pos_target = next(iter(test_dataloader))
pos_hrt, pos_target = pos_hrt.long(), pos_target.float()
pos_hrt, pos_target = pos_hrt.to(device), pos_target.to(device)
head_emb, rel_emb, tail_emb = model.get_embeddings(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])

In [15]:
print(head_emb.shape, rel_emb.shape, tail_emb.shape)
print(head_emb, rel_emb, tail_emb)

torch.Size([1024, 128]) torch.Size([1024, 128]) torch.Size([1024, 128])
tensor([[-3.5075e-08,  1.4886e-08,  4.9449e-08,  ...,  2.4516e-08,
          8.8256e-09, -8.8739e-10],
        [-5.1392e-08,  2.1621e-08,  1.9248e-08,  ...,  2.5856e-08,
          6.7141e-08, -1.7149e-08],
        [ 1.8038e-08, -4.3188e-08,  3.4360e-08,  ..., -4.8237e-08,
          3.9831e-08, -2.4388e-08],
        ...,
        [ 2.0910e-08,  2.1317e-08, -3.1947e-08,  ..., -1.1974e-09,
         -4.6361e-08, -1.0598e-08],
        [ 4.0297e-08, -3.8707e-08, -1.4418e-08,  ..., -2.9823e-08,
          3.4802e-08,  5.8962e-08],
        [ 5.2300e-08, -8.9901e-09,  7.7820e-09,  ...,  8.9428e-09,
          1.3770e-08, -3.2088e-08]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>) tensor([[-2.6009e-08, -6.7773e-07, -4.5543e-07,  ...,  2.1239e-07,
          3.9273e-07,  6.0972e-07],
        [-2.6009e-08, -6.7773e-07, -4.5543e-07,  ...,  2.1239e-07,
          3.9273e-07,  6.0972e-07],
        [-2.6009e-08, -6.7773e-07, -

In [16]:
plau_score = model.get_plausibility_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
print(plau_score)

tensor([-1.2898e-22, -3.9098e-21,  3.4631e-21,  ...,  6.5781e-21,
         4.1331e-21,  2.7280e-21], device='cuda:0', grad_fn=<SumBackward1>)


In [17]:
conf_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
print(conf_score)

tensor([0.5029, 0.5029, 0.5029,  ..., 0.5029, 0.5029, 0.5029], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [18]:
for m in model.fc:
    print(m)
    if isinstance(m, nn.Linear):
        print(m.weight)
        print(m.bias)

Linear(in_features=1, out_features=32, bias=True)
Parameter containing:
tensor([[-4.1084e-07],
        [ 9.9524e-06],
        [ 5.7262e-06],
        [ 4.2099e-06],
        [ 7.4033e-07],
        [ 9.3794e-07],
        [ 1.7766e-06],
        [-6.3830e-07],
        [-1.9440e-08],
        [-4.8341e-08],
        [ 1.8182e-06],
        [-4.1914e-05],
        [-7.9969e-07],
        [-2.8493e-04],
        [ 2.4514e-06],
        [ 4.7232e-04],
        [-2.8363e-07],
        [-2.7865e-06],
        [-4.3242e-06],
        [ 4.4750e-07],
        [ 2.9845e-06],
        [ 4.2299e-04],
        [-8.9398e-07],
        [ 1.4803e-06],
        [ 5.1337e-07],
        [ 9.2689e-07],
        [-7.6715e-07],
        [ 2.9338e-05],
        [-2.5830e-06],
        [-1.7657e-05],
        [ 1.9412e-04],
        [ 5.8059e-07]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([2.5068e-01, 1.4016e-02, 1.2644e-02, 1.2878e-02, 1.4228e-02, 4.9826e-04,
        2.3477e-01, 4.1111e-01, 1.5009e-02, 1.2508e-